In [ ]:
import pandas as pd

facemocap_df = pd.read_pickle("facemocap_df-4.pkl")
facemocap_df.sample(3)

In [1]:
import os
import tkinter as tk
from tkinter import filedialog

def select_directory():
    root = tk.Tk()
    root.withdraw()  # Hide the main window.
    directory_path = filedialog.askdirectory()  # Show the ask directory dialog.
    folder_name = directory_path.split('/')[-1]  # Extract the folder name.
    print(f"Selected Folder: {folder_name}")
    return directory_path

filepath = select_directory()
print(filepath)
# Find the folders in filepath
folders = [f for f in os.listdir(filepath) if os.path.isdir(os.path.join(filepath, f))]

Selected Folder: VJ
C:/Users/ederro/Documents/GitHub/test_3d/VJ


In [2]:
folders

['20230515']

In [7]:
import pandas as pd
import numpy as np
import open3d as o3d    
import seqpc

# Add a new row to the bottom of the dataframe
#facemocap_df = facemocap_df.append({"participant_id": participant_id}, ignore_index=True)

diagnosis = "Facial palsy"
repetitive = False
experimentator = "Félix"
mocap_platform = "IFF"
gender = "F"
bday = "1978-10-16" # YYYY-MM-DD
protocol_name = "SIMOVI 2"
protocol_id = "ID-RCB 2016-A00716-45"
protocol_id_clinicaltrialsgov = "NCT 03115203"
first_name = "Varin"
last_name = "Jennifer"

In [9]:
birthdate = pd.to_datetime(bday, format="%Y-%m-%d")
# Search for the public id of the participant according to its participant_id
participant_id = filepath
public_id = facemocap_df[facemocap_df["Participant ID"] == participant_id]["Public ID"].values[0]
# If the participant is not in the dataframe, add it
if public_id == None:
    public_id = facemocap_df["Public ID"].max() + 1
    
for folder in folders:
    # Change the date from string (YYYYMMDD) to pandas datetime
    date_of_aquisition = pd.to_datetime(folder, format="%Y%m%d")
    age_at_aquisition = date_of_aquisition - birthdate
    # Round age to years
    age_at_aquisition = int(round(age_at_aquisition.days/365.25, 1))
    #participant_id = filepath
    # Find the .csv files in the folder
    files = [f for f in os.listdir(os.path.join(filepath, folder)) if f.endswith(".csv")]
    # Loop through the files
    for file in files:
        if file[-9:-7] == 'M1':
            M=1
        elif file[-9:-7] == 'M2':
            M=2
        elif file[-9:-7] == 'M3':
            M=3
        elif file[-9:-7] == 'M4':
            M=4
        elif file[-9:-7] == 'M5':
            M=5
        else:
            continue
        # Read the .csv file
        df = pd.read_csv(os.path.join(filepath, folder, file), header=None, usecols=range(2,326), skiprows=5)
        spc = df.values.reshape(-1, 108, 3)
        spc = seqpc.impute_missing_values(spc)
        spc_dental_list = []
        for pc in spc:
            pc_dental = seqpc.change_reference_frame(pc, pc[:3])
            spc_dental_list.append(pc_dental)
        spc_dental = np.array(spc_dental_list)
        print(spc_dental.shape)
        if np.isnan(spc_dental).any():
            print("NaN values found in point cloud")

        participant_dict = {"Participant ID": participant_id, "Diagnosis": diagnosis, "File name": file, "Age at Acquisition": age_at_aquisition,
                                            "Birthday": birthdate, "Gender": gender, "First Name": first_name, "Last Name": last_name, "SPC": None, "SPC dental": None,
                                            "Protocol name":protocol_name, "Protocol ID": protocol_id, "Movement": M, "Date of Acquisition": date_of_aquisition,
                                            "Protocol ID clinicaltrials.gov": protocol_id_clinicaltrialsgov, "Repetitive": repetitive,
                                            "Public ID":public_id, "Experimentator": experimentator, "MoCap platform": mocap_platform}
        participant_df = pd.DataFrame(participant_dict, index=[0])
        participant_df.at[0, "SPC"] = spc.reshape(-1,324)
        participant_df.at[0, "SPC dental"] = spc_dental.reshape(-1,108,3)
        facemocap_df.loc[len(facemocap_df)] = participant_df.loc[0]
 

(271, 108, 3)
(314, 108, 3)
(197, 108, 3)
(185, 108, 3)
(293, 108, 3)


In [ ]:
facemocap_df.tail(5)

In [11]:
facemocap_df.to_pickle("facemocap_df-4.pkl")

In [12]:
# Exclude SPC, SPC dental, and SPC imputed from the dataframe
facemocap_df = facemocap_df.drop(["SPC", "SPC dental", "SPC imputed"], axis=1)
facemocap_df.to_csv("facemocap_df-4.csv", index=False)